# Build Initial Dataset 

Example of how to build the initial investing dataset


### 0. Importing Necessary Libs

In [1]:
import Extractors.Extract as methods
from pathlib import Path
import pandas as pd 

data_path = "./DATA"
path = Path(data_path)

### 1. First Build Fundamentus data

In [1]:
methods.build_fundamentos(data_path)

In [2]:
fundamentos = pd.read_csv(Path(data_path) / "fundamentos.csv")

In [3]:
assets = methods.build_full_assets(data_path, fundamentos)

In [4]:
grouping = assets.groupby('Codigo')['P/L'].sum().reset_index()
not_found_codes = grouping[grouping['P/L'] == 0]['Codigo']

In [5]:
teste = methods.get_fundamentos_simple()
teste['Codigo'] = teste.index.str[:4]
not_found_tickers = teste[teste['Codigo'].isin(not_found_codes)].index
not_found_codes = [x for x in not_found_codes if x not in teste['Codigo'].ravel()]

In [6]:
new_fundamentos = methods.build_fundamentos_tickers(data_path, not_found_tickers)

  0%|          | 0/65 [00:00<?, ?it/s]Building Fundamentus data

  2%|▏         | 1/65 [00:03<04:10,  3.92s/it]
error in trying to get data from ticker NTCO3
  5%|▍         | 3/65 [00:27<09:07,  8.83s/it]
error in trying to get data from ticker CEAB3
  9%|▉         | 6/65 [01:00<09:07,  9.29s/it]
error in trying to get data from ticker COGN3
 12%|█▏        | 8/65 [01:15<07:38,  8.05s/it]
error in trying to get data from ticker GPIV33
 14%|█▍        | 9/65 [01:19<06:18,  6.75s/it]
error in trying to get data from ticker ATMP3
 18%|█▊        | 12/65 [01:33<04:37,  5.23s/it]
error in trying to get data from ticker AVLL3
 20%|██        | 13/65 [01:37<04:11,  4.84s/it]
error in trying to get data from ticker CASH3
 22%|██▏       | 14/65 [01:40<03:49,  4.50s/it]
error in trying to get data from ticker CURY3
 23%|██▎       | 15/65 [01:44<03:38,  4.38s/it]
error in trying to get data from ticker DMVF3
 25%|██▍       | 16/65 [01:48<03:24,  4.17s/it]
error in trying to get data from ticker ENJU3

In [7]:
fundamentos = fundamentos.append(new_fundamentos)
fundamentos.to_csv(Path(data_path) / "fundamentos.csv", index = False)
assets = methods.build_full_assets(data_path, fundamentos)

In [9]:
with open(Path(data_path) / 'log.txt', 'r') as f:
    log = f.readlines()
log = [x.replace("\n", "") for x in log]

In [49]:
renaming = {"Cotacao" : "Última Cotação ON",
"P/L" : "P/L",
"P/VP" : "P/VPA",
"PSR" : "PSR",
"DY" : "DY",
"P/Ativo" : "P/Ativo",
"P/Cap.Giro" : 'P/Capital de Giro',
"P/EBIT" : 'P/EBIT',
"P/ACL" : 'P/ACL',
"EV/EBIT" : 'EV/EBIT',
"EV/EBITDA" : "REMOVER",
"Mrg.Ebit" : 'Margem EBIT',
"Mrg.Liq." : "REMOVER",
"Liq.Corr." : "REMOVER",
"ROIC" : "REMOVER",
"ROE" : "ROE",
"Liq.2meses" : "REMOVER",
"Pat.Liq" : "REMOVER",
"Div.Brut/Pat." : "REMOVER",
"Cresc.5anos" : "REMOVER",
"Codigo" : "Codigo" }

In [48]:
teste['Ticker'] = teste.index
teste = teste.reset_index()
teste = teste[teste['Ticker'].isin(log)]
teste = teste.rename(columns = renaming)
teste = teste.drop(columns = "REMOVER")
teste = teste.drop(columns = 'index')
teste['Data'] = ["2020"] * len(teste)

In [68]:
fundamentos = fundamentos.append(teste)
fundamentos.to_csv(Path(data_path) / "fundamentos.csv", index = False)
assets = methods.build_full_assets(data_path, fundamentos)

### 2. Extract Macro Price data 

Extracting 2 years of data for every ticker in assets database. This data will be daily. This will be used only for extracting some metrics. More precise data will be extract later to use in ML models.

In [5]:
assets = pd.read_csv(path / "assets.csv")
tickers = assets['Ticker'].unique()

In [6]:
history = methods.build_price_history(data_path, tickers)

In [7]:
history = pd.read_csv(path / "history.csv")

In [9]:
history['symbol'].nunique()

464

### 3. Extract Cripto data 

Extracting 2 years of data for every ticker in assets database. This data will be daily. This will be used only for extracting some metrics. More precise data will be extract later to use in ML models.

In [2]:
criptos = pd.read_csv(Path("./Extractors/cripto/digital_currency_list.csv"))

In [ ]:
history = methods.build_cripto_history(data_path, criptos['currency code'].unique().ravel())